In [32]:
import pandas as pd
from sklearn.metrics.cluster import v_measure_score
import random
from sklearn.cluster import AgglomerativeClustering
import numpy as np

In [33]:
random.seed(42)

In [34]:
dataset_name = 'breast_cancer_coimbra'
dataset_test_path = f'./data/{dataset_name}-train.csv'
dataset_train_path = f'./data/{dataset_name}-test.csv'
df_train = pd.read_csv(dataset_train_path)
df_test = pd.read_csv(dataset_test_path)

X_train = df_train.drop(columns=['Classification']).to_numpy()
y_train = df_train['Classification'].to_numpy()
X_test = df_test.drop(columns=['Classification']).to_numpy()
y_test = df_test['Classification'].to_numpy()

In [35]:
MIN_CONST = 1
MAX_CONST = 5
NAO_TERMINAL_F = [lambda x, a, b : pow(x, a/b), lambda x, a, b : x * (a/b)]
NUM_F = len(NAO_TERMINAL_F)
NAO_TERMINAL_A = [1,2,3,4,5]
NUM_A = len(NAO_TERMINAL_A)
class Individuo():
    def __init__(self,  n_features, genotipo = None, genotipo_estruturado = None):
        self._n_features = n_features
        self._genotipo = genotipo
        if(genotipo_estruturado):
            self._genotipo_estruturado = genotipo_estruturado
            return
        self._genotipo_estruturado = []
        for i in range(0, len(genotipo), 3):
            num1, num2, num3 = genotipo[i:i+3]
            self._genotipo_estruturado.append([[num1 % NUM_F], [num2 % NUM_A, num3 % NUM_A]])
    
    def get_fenotipo(self):
        fenotipo = []
        for i in range(self._n_features):
            f = NAO_TERMINAL_F[self._genotipo_estruturado[i][0][0]]
            a = NAO_TERMINAL_A[self._genotipo_estruturado[i][1][0]]
            b = NAO_TERMINAL_A[self._genotipo_estruturado[i][1][1]]
            fenotipo.append(lambda x : f(x, a, b))
        self._fenotipo = fenotipo
        return fenotipo
    
    def fazer_mutacao(self):
        random_index = random.randint(0, self._n_features - 1)
        self._genotipo_estruturado[random_index][0][0] = random.randint(0, NUM_F - 1)
        self._genotipo_estruturado[random_index][1][0] = random.randint(0, NUM_A - 1)
        self._genotipo_estruturado[random_index][1][1] = random.randint(0, NUM_A - 1)
    
    def calcular_distancia(self, xi, xj):
        diff = abs(xi - xj)
        sum = 0
        for i in range(self._n_features):
            sum += self.get_fenotipo()[i](diff[i])
        return sum


    def fitness(self, X, y):
        n_instancias = len(X)
        n_labels = len(set(y))
        # Criar np matriz de n_instancias x n_instancias
        matriz_distancias = np.zeros((n_instancias, n_instancias))
        for i in range(n_instancias):
            for j in range(n_instancias):
                distancia = self.calcular_distancia(X[i], X[j])
                matriz_distancias[i][j] = matriz_distancias[j][i] = distancia
        clustering = AgglomerativeClustering(
        n_clusters=n_labels,   
        metric='precomputed', 
        linkage='complete' 
        )
        labels = clustering.fit_predict(matriz_distancias)
        return v_measure_score(y, labels) 

In [36]:
bd = bytes(random.getrandbits(8) for _ in range(3 * 9))
print(bd[0])

163


In [37]:
binary_data = bytes(random.getrandbits(8) for _ in range(3 * 9))
i = Individuo(9, genotipo = binary_data)
print(i.get_fenotipo()[0](2))
i.fazer_mutacao()
i.get_fenotipo()[0](2)

2.5198420997897464


2.5198420997897464

In [38]:
class Populacao():
    def __init__(self, n_pop, pc, pm, tam_torneio, X, y, eh_elitismo = False):
        self._populacao = []
        self._n_pop = n_pop
        self._pc = pc
        self._pm = pm
        self._tam_torneio = tam_torneio
        self._eh_elitismo = eh_elitismo
        self._X = X
        self._y = y
        self._n_features = X.shape[1]
        self.gerar_pop_inicial()
    
    def gerar_pop_inicial(self):
        for i in range(self._n_pop):
            genotipo = bytes(random.getrandbits(8) for _ in range(3 * self._n_features))
            ind = Individuo(n_features=self._n_features, genotipo=genotipo)
            self._populacao.append((ind, ind.fitness(X_train, y_train)))
    
    def cruzamento(self, pai1, pai2):
        range_pai1 = random.randint(1, self._n_features - 1)
        genotipo_estruturado_filho = []
        for i in range(self._n_features):
            if(i <= range_pai1):
                genotipo_estruturado_filho.append(pai1._genotipo_estruturado[i])
            else:
                genotipo_estruturado_filho.append(pai2._genotipo_estruturado[i])
        return Individuo(n_features = self._n_features, genotipo_estruturado = genotipo_estruturado_filho)


    def escolher_pais(self):
        pai1 = self._populacao[random.randint(0, self._n_pop - 1)][0]
        while True:
            pai2 = self._populacao[random.randint(0, self._n_pop - 1)][0]
            if pai1 != pai2:
                break
        return pai1, pai2

    def gerar_filhos(self):
        n_filhos = 0
        while(n_filhos < self._n_pop):
            pai1, pai2 = self.escolher_pais()
            if(random.random() < self._pc):
                filho = self.cruzamento(pai1, pai2)
            else:
                filho = pai1
            if(random.random() < self._pm):
                filho.fazer_mutacao()
            self._populacao.append((filho, filho.fitness(X_train, y_train)))
            n_filhos += 1

        
    def fazer_selecao(self):
        nova_pop = []
        self._populacao.sort(key = lambda x : x[1], reverse = True)
        if(self._eh_elitismo):
            nova_pop.append(self._populacao[0])
        while(len(nova_pop) < self._n_pop):
            torneio = []
            while(len(torneio) < self._tam_torneio):
                indice_aleatorio = random.randint(0, self._n_pop - 1)
                if(self._populacao[indice_aleatorio] not in torneio):
                    torneio.append(self._populacao[indice_aleatorio])
            torneio.sort(key = lambda x : x[1], reverse = True)
            nova_pop.append(torneio[0])
        self._populacao = nova_pop
        self._populacao.sort(key = lambda x : x[1], reverse = True)
    
    def get_fitness_media(self):
        return sum([x[1] for x in self._populacao]) / self._n_pop
    
    def get_melhor_individuo(self):
        return self._populacao[0]

    

In [39]:
N_GERACOES = 50
N_POP = 100
PC = 0.9
PM = 0.05
TAM_TORNEIO = 5
EH_ELITISMO = False
POP = Populacao(N_POP, PC, PM, TAM_TORNEIO, X_train, y_train, eh_elitismo = EH_ELITISMO)
for i in range(N_GERACOES):
    POP.gerar_filhos()
    POP.fazer_selecao()
    print(f'Geração {i} - Fitness médio: {POP.get_fitness_media()} - Melhor fitness: {POP.get_melhor_individuo()[1]}')



Geração 0 - Fitness médio: 0.13730404769080573 - Melhor fitness: 0.17871023341808764
Geração 1 - Fitness médio: 0.17605599074326161 - Melhor fitness: 0.17871023341808764
Geração 2 - Fitness médio: 0.17871023341808734 - Melhor fitness: 0.17871023341808764
Geração 3 - Fitness médio: 0.17871023341808734 - Melhor fitness: 0.17871023341808764
Geração 4 - Fitness médio: 0.17871023341808734 - Melhor fitness: 0.17871023341808764
Geração 5 - Fitness médio: 0.17871023341808734 - Melhor fitness: 0.17871023341808764
Geração 6 - Fitness médio: 0.17871023341808734 - Melhor fitness: 0.17871023341808764
Geração 7 - Fitness médio: 0.17871023341808734 - Melhor fitness: 0.17871023341808764
Geração 8 - Fitness médio: 0.17871023341808734 - Melhor fitness: 0.17871023341808764
Geração 9 - Fitness médio: 0.17871023341808734 - Melhor fitness: 0.17871023341808764
Geração 10 - Fitness médio: 0.17871023341808734 - Melhor fitness: 0.17871023341808764
Geração 11 - Fitness médio: 0.17871023341808734 - Melhor fitness